In [1]:
import pandas as pd
import ast
from neo4j import GraphDatabase
from statistics import mode

In [5]:
extracted = pd.read_csv(r'C:\Users\suyog\Desktop\monsoon_24\capstone-legal-docs-analysis\openai_extracted.csv', index_col=0)

In [6]:
extracted['statutes'] = extracted['statutes'].apply(lambda x: ast.literal_eval(x))
extracted['provisions'] = extracted['provisions'].apply(lambda x: ast.literal_eval(x))

In [7]:
extracted

,file_name,summary,court,petitioners,respondents,judges,date,org,gpe,provisions,statutes,precedents,key_facts,type_of_case,decision
0,Abhijeet_Suryakant_Maske_And_Anr_vs_The_State_...,In the case of Abhijeet Suryakant Maske and An...,HIGH COURT OF JUDICATURE AT BOMBAY,"['Abhijeet Suryakant Maske', 'Dilip Ramchandra...",['The State of Maharashtra'],['C.V. Bhadang'],01-03-2022,[],"['Pune', 'Maharashtra']","[Section 328 of IPC, Section 272 of IPC, Secti...","[Food Safety and Standards Act, 2006]","['Joseph Kurian & Anr. Vs. State of Kerala', '...",The Applicants are seeking anticipatory bail i...,Anticipatory Bail Application,in favour of appellant
1,Amluya_Anand_vs_State_on_8_March_2022.PDF,**Case Summary: Amluya Anand vs State (8 March...,HIGH COURT OF JUDICATURE FOR RAJASTHAN AT JODHPUR,['Amluya Anand'],"['State, Through Vinod Sharma, Food Safety Off...",['Sandeep Mehta'],08-03-2022,"['Amul Dairy', 'Food Safety Officer', 'Chief M...","['Mujjafarpur', 'Gurgaon', 'Nagaur', 'Devgarh ...","[Section 468 of Cr.P.C., Section 77 of the Foo...","[Food Safety and Standards Act, 2006]",['Sri.V.V.S.S.R. Prakash Rao vs. The State of ...,The petitioner sought to quash orders taking c...,Criminal Miscellaneous Petition,in favour of appellant
2,Arulmurugan_Starch_Industries_vs_Food_Safety_A...,**Case Summary: Arulmurugan Starch Industries ...,High Court of Judicature at Madras,['Arulmurugan Starch Industries'],['Food Safety and Standards Authority of India...,['Krishnan Ramasamy'],19-01-2022,['Food Safety and Standards Authority of India...,"['Salem District', 'New Delhi', 'Chennai']",[Article 226 of the Constitution of India],"[Food Safety and Standards Act, 2006]",[],The petitioner filed a writ petition challengi...,Writ Petition,The court directed the Commissioner of Food Sa...
3,Arun_Raosaheb_Khot_vs_State_Of_Maharashtra_on_...,In the case of Arun Raosaheb Khot vs. State of...,High Court of Judicature at Bombay,['Arun Raosaheb Khot'],['The State of Maharashtra'],['C.V. Bhadang'],31-01-2022,[],"['Pune', 'Maharashtra']","[Section 272 of IPC, Section 273 of IPC, Secti...","[Food Safety and Standards Act, 2006]",['Joseph Kurian & Anr. Vs. State of Kerala'],The applicant is seeking anticipatory bail in ...,Anticipatory Bail Application,in favour of appellant
4,Asha_Singh_vs_The_State_Of_Bihar_on_24_March_2...,In the case of Asha Singh vs The State of Biha...,HIGH COURT OF JUDICATURE AT PATNA,['Asha Singh'],"['The State of Bihar', 'THE UNION OF INDIA THR...",['Ashutosh Kumar'],24-03-2022,"['Tata Institute of Social Sciences', 'SAKHI']","['Bihar', 'Patna', 'East Champaran', 'Motihari']","[Section 341 of IPC, Section 342 of IPC, Secti...","[Indian Penal Code, Protection of Children fro...",[],"The petitioner, Asha Singh, is seeking anticip...",Criminal Miscellaneous,against appellant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Vijaysing_Dnyaneshwar_Gotya_S_O_vs_State_Of_Mh...,**Case Summary: Vijaysing @ Dnyaneshwar @ Goty...,"High Court of Judicature at Bombay, Nagpur Bench",['Vijaysing @ Dnyaneshwar @ Gotya Raghunandan ...,['State of Maharashtra'],['Avinash G. Gharote'],13-01-2022,[],"['Maharashtra', 'Mumbai']","[Section 188 of IPC, Section 272 of IPC, Secti...","[Food Safety and Standard Act 2006, Cigarettes...",['Ganesh Pandurang Jadhao vrs. State of Mahara...,The applicant was arrested for transporting co...,Criminal Application (Bail),against appellant
192,Viky_Vikash_Ku_Agrawal_vs_State_Of_Odisha_Opp_...,In the case of Viky @ Vikash Ku Agrawal vs Sta...,High Court of Orissa at Cuttack,['Viky @ Vikash Ku Agrawal'],['State of Odisha'],['S.K. Sahoo'],09-02-2022,[],"['Odisha', 'Kantabanji']","[Section 438 of Cr.P.C., Section 272 of IPC, S...","[Indian Penal Code, Food Safety and Standards ...",[],The petitioner applied for anticipatory bail i...,Anticipatory bail application,in favour of appellant
193,Vishnu_Gupta_Lalla_vs_The_State_Of_Madhya_Prad...,"On January 3, 2022, the High Court of Madhya P...",High Court Of Madhy

In [8]:
all_statutes = []
for statutes in extracted['statutes']:
    for statute in statutes:
        all_statutes.append(statute)


In [9]:
all_statutes.sort()
all_statutes

['A.P. Excise Act',
 'A.P. Excise Act',
 'A.P. Excise Act',
 'A.P. Excise Act',
 'A.P. Excise Act',
 'A.P. Excise Act',
 'A.P. Excise Amendment Act, 2020',
 'A.P. Gaming Act',
 'APPE Act',
 'Andhra Pradesh (Andhra Area) Public Health Act, 1939',
 'Andhra Pradesh Excise Act, 1968',
 'Andhra Pradesh Excise Act, 1968',
 'Andhra Pradesh Excise Act, 1968',
 'Arbitration & Conciliation Act, 1996',
 'Arbitration & Conciliation Act, 1996',
 'Arbitration and Conciliation Act, 1996',
 'Armed Forces (Special Powers) Act, 1958',
 'Assam Excise (Amendment) Act, 2018',
 'Bihar Finance (Amendment) Rules, 2005',
 'Bihar Prohibition and Excise Act, 2016',
 'Bihar Prohibition and Excise Act, 2016',
 'Bihar Prohibition and Excise Act, 2016',
 'Bihar Prohibition and Excise Act, 2016',
 'Bonded Labour System (Abolition) Act, 1976',
 'CORPA Act',
 'COTPA Act',
 'Child and Adolescent Labour (Prohibition and Regulation) Act, 1986',
 'Cigarettes and Other Tobacco Products (Prohibition of Advertisement and Regu

In [10]:
from neo4j import GraphDatabase, basic_auth


In [11]:
driver = GraphDatabase.driver(
  "bolt://localhost:7687",
  auth=basic_auth("neo4j", "password"))

In [12]:
def create_graph(tx, pdf_file, statute, provision):
    query = (
        "MERGE (p:PDF {name: $pdf_file}) "
        "MERGE (s:Statute {name: $statute}) "
        "MERGE (r:Provision {name: $provision}) "
        "MERGE (p)-[:CITES]->(r)"
        "MERGE (r)-[:PROVISION_IN]->(s)"
    )
    tx.run(query, pdf_file=pdf_file, statute=statute, provision=provision)

def create_graph_without_prov(tx, pdf_file, statute):
    query = (
        "MERGE (p:PDF {name: $pdf_file}) "
        "MERGE (s:Statute {name: $statute}) "
        "MERGE (p)-[:CITES]->(s)"
    )
    tx.run(query, pdf_file=pdf_file, statute=statute)



In [19]:
extracted

,file_name,summary,court,petitioners,respondents,judges,date,org,gpe,provisions,statutes,precedents,key_facts,type_of_case,decision
0,Abhijeet_Suryakant_Maske_And_Anr_vs_The_State_...,In the case of Abhijeet Suryakant Maske and An...,HIGH COURT OF JUDICATURE AT BOMBAY,"['Abhijeet Suryakant Maske', 'Dilip Ramchandra...",['The State of Maharashtra'],['C.V. Bhadang'],01-03-2022,[],"['Pune', 'Maharashtra']","[Section 328 of IPC, Section 272 of IPC, Secti...","[Food Safety and Standards Act, 2006]","['Joseph Kurian & Anr. Vs. State of Kerala', '...",The Applicants are seeking anticipatory bail i...,Anticipatory Bail Application,in favour of appellant
1,Amluya_Anand_vs_State_on_8_March_2022.PDF,**Case Summary: Amluya Anand vs State (8 March...,HIGH COURT OF JUDICATURE FOR RAJASTHAN AT JODHPUR,['Amluya Anand'],"['State, Through Vinod Sharma, Food Safety Off...",['Sandeep Mehta'],08-03-2022,"['Amul Dairy', 'Food Safety Officer', 'Chief M...","['Mujjafarpur', 'Gurgaon', 'Nagaur', 'Devgarh ...","[Section 468 of Cr.P.C., Section 77 of the Foo...","[Food Safety and Standards Act, 2006]",['Sri.V.V.S.S.R. Prakash Rao vs. The State of ...,The petitioner sought to quash orders taking c...,Criminal Miscellaneous Petition,in favour of appellant
2,Arulmurugan_Starch_Industries_vs_Food_Safety_A...,**Case Summary: Arulmurugan Starch Industries ...,High Court of Judicature at Madras,['Arulmurugan Starch Industries'],['Food Safety and Standards Authority of India...,['Krishnan Ramasamy'],19-01-2022,['Food Safety and Standards Authority of India...,"['Salem District', 'New Delhi', 'Chennai']",[Article 226 of the Constitution of India],"[Food Safety and Standards Act, 2006]",[],The petitioner filed a writ petition challengi...,Writ Petition,The court directed the Commissioner of Food Sa...
3,Arun_Raosaheb_Khot_vs_State_Of_Maharashtra_on_...,In the case of Arun Raosaheb Khot vs. State of...,High Court of Judicature at Bombay,['Arun Raosaheb Khot'],['The State of Maharashtra'],['C.V. Bhadang'],31-01-2022,[],"['Pune', 'Maharashtra']","[Section 272 of IPC, Section 273 of IPC, Secti...","[Food Safety and Standards Act, 2006]",['Joseph Kurian & Anr. Vs. State of Kerala'],The applicant is seeking anticipatory bail in ...,Anticipatory Bail Application,in favour of appellant
4,Asha_Singh_vs_The_State_Of_Bihar_on_24_March_2...,In the case of Asha Singh vs The State of Biha...,HIGH COURT OF JUDICATURE AT PATNA,['Asha Singh'],"['The State of Bihar', 'THE UNION OF INDIA THR...",['Ashutosh Kumar'],24-03-2022,"['Tata Institute of Social Sciences', 'SAKHI']","['Bihar', 'Patna', 'East Champaran', 'Motihari']","[Section 341 of IPC, Section 342 of IPC, Secti...","[Indian Penal Code, Protection of Children fro...",[],"The petitioner, Asha Singh, is seeking anticip...",Criminal Miscellaneous,against appellant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Vijaysing_Dnyaneshwar_Gotya_S_O_vs_State_Of_Mh...,**Case Summary: Vijaysing @ Dnyaneshwar @ Goty...,"High Court of Judicature at Bombay, Nagpur Bench",['Vijaysing @ Dnyaneshwar @ Gotya Raghunandan ...,['State of Maharashtra'],['Avinash G. Gharote'],13-01-2022,[],"['Maharashtra', 'Mumbai']","[Section 188 of IPC, Section 272 of IPC, Secti...","[Food Safety and Standard Act 2006, Cigarettes...",['Ganesh Pandurang Jadhao vrs. State of Mahara...,The applicant was arrested for transporting co...,Criminal Application (Bail),against appellant
192,Viky_Vikash_Ku_Agrawal_vs_State_Of_Odisha_Opp_...,In the case of Viky @ Vikash Ku Agrawal vs Sta...,High Court of Orissa at Cuttack,['Viky @ Vikash Ku Agrawal'],['State of Odisha'],['S.K. Sahoo'],09-02-2022,[],"['Odisha', 'Kantabanji']","[Section 438 of Cr.P.C., Section 272 of IPC, S...","[Indian Penal Code, Food Safety and Standards ...",[],The petitioner applied for anticipatory bail i...,Anticipatory bail application,in favour of appellant
193,Vishnu_Gupta_Lalla_vs_The_State_Of_Madhya_Prad...,"On January 3, 2022, the High Court of Madhya P...",High Court Of Madhy

In [20]:
with driver.session() as session:
    done_files = []
    for index, row in extracted.iterrows():
        pdf_file = row['file_name']
        if pdf_file in done_files:
            pdf_file = pdf_file + '_copy'
        done_files.append(pdf_file)
        statutes = row['statutes']  # Convert string representation of list back to list
        provisions = row['provisions']
        
        for statute in statutes:
            session.write_transaction(create_graph_without_prov, pdf_file, statute)
        
        # if len(provisions) < 1:
        #     for statute in statutes:
        #         session.write_transaction(create_graph_without_prov, pdf_file, statute)
        # if len(provisions) <= len(statutes) and len(provisions) > 0:
        #     #write to graph, matching the n provisions with the first n statutes, then write to graph without provisions the remaining statutes
        #     for i in range(len(provisions)):
        #         print(pdf_file, statutes[i], provisions[i])
        #         session.write_transaction(create_graph, pdf_file, statutes[i], provisions[i])
        #     for i in range(len(provisions), len(statutes)):
        #         session.write_transaction(create_graph_without_prov, pdf_file, statutes[i])
        
    print(index)


C:\Users\suyog\AppData\Local\Temp\ipykernel_120096\3237909036.py:12: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_graph_without_prov, pdf_file, statute)


195
